In [1]:
# import library
import pymoo
from pymoo.model.problem import Problem
import pandas
import pycaret
import numpy as np
import math
import matplotlib.pyplot as plt

from pycaret.regression import load_model
#

In [2]:
#"ML_model2_v3_2021_08_12_v1_Lmr"
ML_model_Lmt = load_model("./model_2021_08_17_v1/ML_model2_v3_2021_08_17_v1_Lmt") # Lmt model
ML_model_Lmr = load_model("./model_2021_08_17_v1/ML_model2_v3_2021_08_17_v1_Lmr") # Lmr model
ML_model_Llt = load_model("./model_2021_08_17_v1/ML_model2_v3_2021_08_17_v1_Llt") # Llt model
ML_model_Llr = load_model("./model_2021_08_17_v1/ML_model2_v3_2021_08_17_v1_Llr") # Llr model
ML_model_R1 = load_model("./model_2021_08_17_v1/ML_model2_v3_2021_08_17_v1_R1") # R1 model
ML_model_R2 = load_model("./model_2021_08_17_v1/ML_model2_v3_2021_08_17_v1_R2") # R2 model

ML_model_Lmtc = load_model("ML_model2_v3_2021_08_12_v1_Lmt_c") # Lmt model
ML_model_Lmrc = load_model("ML_model2_v3_2021_08_12_v1_Lmr_c") # Lmr model
ML_model_Lltc = load_model("ML_model2_v3_2021_08_12_v1_Llt_c") # Llt model
ML_model_Llrc = load_model("ML_model2_v3_2021_08_12_v1_Llr_c") # Llr model
ML_model_R1c = load_model("ML_model2_v3_2021_08_12_v1_R1_c") # R1 model
ML_model_R2c = load_model("ML_model2_v3_2021_08_12_v1_R2_c") # R2 model

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [ ]:
filename = "Data_2021_08_13_v1_m.csv" 
csv = pandas.read_csv(filename)
csv = csv.dropna(axis=0).reset_index(drop=True)#drop NaN

csv = csv[csv['Lmt']>0.1].reset_index(drop=True)
csv = csv[csv['Lmr']>0.1].reset_index(drop=True)
csv = csv[csv['k']>0].reset_index(drop=True)



def l2pp(lst):
    return pandas.DataFrame(lst,columns=["N1","N2","d1","d2","freq","move_tx","move_rx","offset_tx","offset_rx","per","space1","space2","space3","space4","space5","space6","l1", "l2", "h1", "w1", "I1", "I2","N1s","N2s","offset","length1","length4","widndow2","from_l2"])


In [ ]:
csv.describe()
list(csv.describe().iloc[2,1:21])

In [ ]:
def l2p(X):

    N1 = X[:,0]
    N2 = X[:,1]

    freq = X[:,4]

    per = X[:,9] * 4 * math.pi * 1e-7

    # unit [m]
    d1 = X[:,2]
    d2 = X[:,3]

    move_tx = X[:,5]
    move_rx = X[:,6]

    offset_tx = X[:,7]
    offset_rx = X[:,8]

    space1 = X[:,10]
    space2 = X[:,11] 
    space3 = X[:,12]
    space4 = X[:,13] 
    space5 = X[:,14]
    space6 = X[:,15]

    l1 = X[:,16]
    l2 = X[:,17]
    h1 = X[:,18]
    w1 = X[:,19]

    N1s = N1 ** 2
    N2s = N2 ** 2
    offset = abs(offset_tx-offset_rx)
    length1 = N1 * (l1*2 + space1*2 + w1 + space3*2)/2 + N1 * (l1*2 + space1*2 + space5 *2 + w1 + space3*2)/2
    length4 = N2 * (l1*2 + space2*2 + w1 + space4*2)/2 + N2 * (l2*2 + space2*2 + space6 *2 + w1 + space4*2)/2
    window1 = (l1 * 2 + space1 *2) * (w1 + space3 *2)
    window2 = (l1 * 2 + space2 *2) * (w1 + space4 *2)
    from_l2 = l2 - space2 - space5

    X[:,0] = (X[:,0] - csv.describe().iloc[1,:][1])/csv.describe().iloc[2,:][1]
    X[:,0] = (X[:,1] - csv.describe().iloc[1,:][2])/csv.describe().iloc[2,:][2]
    X[:,2] = (X[:,2] - csv.describe().iloc[1,:][3])/csv.describe().iloc[2,:][3]
    X[:,3]= (X[:,3] - csv.describe().iloc[1,:][4])/csv.describe().iloc[2,:][4]
    #csv["freq"] = (csv["freq"] - csv.describe().iloc[1,:][5])/csv.describe().iloc[2,:][5]
    X[:,5] = (X[:,5] - csv.describe().iloc[1,:][6])/csv.describe().iloc[2,:][6]
    X[:,6] = (X[:,6] - csv.describe().iloc[1,:][7])/csv.describe().iloc[2,:][7]
    X[:,7] = (X[:,7] - csv.describe().iloc[1,:][8])/csv.describe().iloc[2,:][8]
    X[:,8] = (X[:,8] - csv.describe().iloc[1,:][9])/csv.describe().iloc[2,:][9]
    X[:,9] = (X[:,9] - csv.describe().iloc[1,:][10])/csv.describe().iloc[2,:][10]
    X[:,10] = (X[:,10] - csv.describe().iloc[1,:][11])/csv.describe().iloc[2,:][11]
    X[:,11] = (X[:,11] - csv.describe().iloc[1,:][12])/csv.describe().iloc[2,:][12]
    X[:,12]  = (X[:,12]  - csv.describe().iloc[1,:][13])/csv.describe().iloc[2,:][13]
    X[:,13]  = (X[:,13]  - csv.describe().iloc[1,:][14])/csv.describe().iloc[2,:][14]
    X[:,14]  = (X[:,14]  - csv.describe().iloc[1,:][15])/csv.describe().iloc[2,:][15]
    X[:,15]  = (X[:,15]  - csv.describe().iloc[1,:][16])/csv.describe().iloc[2,:][16]
    X[:,16]  = (X[:,16]  - csv.describe().iloc[1,:][17])/csv.describe().iloc[2,:][17]
    X[:,17] = (X[:,17]  - csv.describe().iloc[1,:][18])/csv.describe().iloc[2,:][18]
    X[:,18]  = (X[:,18]  - csv.describe().iloc[1,:][19])/csv.describe().iloc[2,:][19]
    X[:,19]  = (X[:,19]  - csv.describe().iloc[1,:][20])/csv.describe().iloc[2,:][20]

    N1s = pandas.DataFrame(N1s,columns=["N1s"])
    N2s = pandas.DataFrame(N2s,columns=["N2s"])
    offset = pandas.DataFrame(offset,columns=["offset"])
    length1 = pandas.DataFrame(length1,columns=["length1"])
    length4 = pandas.DataFrame(length4,columns=["length4"])
    window1 = pandas.DataFrame(window1,columns=["window1"])
    window2 = pandas.DataFrame(window1,columns=["window2"])
    from_l2 = pandas.DataFrame(window1,columns=["from_l2"])

    #X = pandas.DataFrame(lst,columns=["N1","N2","d1","d2","freq","move_tx","move_rx","offset_tx","offset_rx","per","space1","space2","space3","space4","space5","space6","l1", "l2", "h1", "w1", "I1", "I2","N1s","N2s","offset","length1","length4","widndow2","from_l2"])
    X = pandas.DataFrame(X,columns=["N1","N2","d1","d2","freq","move_tx","move_rx","offset_tx","offset_rx","per","space1","space2","space3","space4","space5","space6","l1", "l2", "h1", "w1", "I1", "I2"])
    data = pandas.concat([X,N1s,N2s,offset,length1,length4,window1,window2,from_l2],axis=1)

    return data

In [5]:
#GLOBAL VARS
INPUT_MEANS = [11.52298422039056, 11.456883174462424, 1.250509864774989, 1.2538539126579504, 0, 2.296427822897579, 2.298618931500766, -0.22316660284959997, -0.22387698756574836, 3498.3822575119407, 3.751657665101436, 20.142866427520758, 3.749931581386913, 20.117956611112046, 3.5000719453457, 3.5003074303218167, 13.929453255677942, 42.464752826424295, 52.22957014167389, 59.56812639809758, 5.02519452690933, 4.981609450455427, 33.70174577586959, 32.88914204821819, 0.995559901354316, 5.058896272683628, 5.014498592502798, 4.472254444692369, 58.21624804897175, 57.724760906550095, 0.37850598975131616, 0.3799372194302846, 0.008550744438473683, 0.012024154640011416, 0, 0]
INPUT_STDS = [4.051942143540716, 3.995097310727702, 0.4316337047899765, 0.4294732374156009, 1, 0.6957729777394037, 0.6977392847897756, 9.669547184856619, 9.669288866198748, 864.415584487189, 1.4757152190445155, 5.821673397813458, 1.4760283962164227, 5.824862857828931, 0.8345365926161037, 0.8325621560324485, 3.4611454895181297, 10.451871407488593, 16.258845716118564, 23.037792312255746, 4.547030895189253, 4.568923127681277, 24.953547369232165, 23.935996346040913, 0.0027699777076083173, 4.564224228050054, 4.58539660986043, 3.5122816008824467, 52.61663562827219, 52.86936082709284, 0.20188697414812357, 0.20143747396635506, 0.01081075942750983, 0.015381580408170992, 1, 1]



def preprocess(X, new_feature_names = []):
    #input :        X   : [Batch, 22] shape
    #    new_features   : arr of new feature names 
    #output  Y : [Batch, 29] shape
    #do normalize and add some features

    #Unpack Inputs
    N1,N2,d1,d2,freq,move_tx,move_rx,offset_tx,offset_rx,per,space1,space2,space3,space4,space5,space6,l1, l2, h1, w1, I1, I2 = np.hsplit(X,22)
    input_arrs = [N1,N2,d1,d2,freq,move_tx,move_rx,offset_tx,offset_rx,per,space1,space2,space3,space4,space5,space6,l1, l2, h1, w1, I1, I2] # 22

    #Generae Features
    N1s     = N1 ** 2
    N2s     = N2 ** 2
    offset  = abs(offset_tx-offset_rx)
    length1 = N1 * (l1*2 + space1*2 + w1 + space3*2)/2 + N1 * (l1*2 + space1*2 + space5 *2 + w1 + space3*2)/2
    length4 = N2 * (l1*2 + space2*2 + w1 + space4*2)/2 + N2 * (l2*2 + space2*2 + space6 *2 + w1 + space4*2)/2
    window1 = (l1 * 2 + space1 *2) * (w1 + space3 *2)
    window2 = (l1 * 2 + space2 *2) * (w1 + space4 *2)
    from_l2 = l2 - space2 - space5

    new_features_dict = {
        "N1s"       : N1s,
        "N2s"       : N2s,
        "offset"    : offset,
        "length1"   : length1,
        "length4"   : length4,
        "window1"   : window1,
        "window2"  : window2,
        "from_l2"   : from_l2
    }
    #filter by 'new_features'
    new_features_arr = [v for (k,v) in new_features_dict.items() if k in new_feature_names]

    #Standrize 
    normalized_inp = [ (inp-mean)/std  for i,(inp, mean,std) in enumerate(zip(input_arrs, INPUT_MEANS,INPUT_STDS)) ]

    #Merge
    new_model_input = normalized_inp + new_features_arr
    
    #list to np tensor to pandas df
    new_model_input =  np.concatenate(new_model_input,axis=1)
    return pandas.DataFrame(
        new_model_input,
        columns =
            ["N1","N2","d1","d2","freq","move_tx","move_rx","offset_tx","offset_rx","per","space1","space2","space3","space4","space5","space6","l1", "l2", "h1", "w1", "I1", "I2"] + new_feature_names
        )
    

In [ ]:
csv.describe().iloc[2].tolist()

In [ ]:
ML_model_Lmt

In [ ]:
%timeit  preprocess(a,new_feature_names=["N1s","offset","length1"])

In [ ]:
%timeit ML_model_Lmt.predict(b)

In [6]:
#inputs
#[[n1, n2, space1, space2, l1, l2, h1, w1]]
# n1 = 1~9
# n2 = 1~9
# space1 = 
# 
# core loss parameter
core_temp = 70

core_a = 0.6942
core_x = 1.4472
core_y = 2.4769
core_b = 4.7948
core_c = 0.0684
core_d = 4e-4
core_LT = core_b - core_c*core_temp + core_d*core_temp**2

#move_tx = 3

def get_coreloss(core_a,core_x,core_y,core_LT,V1,freq,N1,Lmt,l1,l2,h1,w1):
    #IMPLEMENT
    B = get_B(V1,freq,Lmt,N1,w1,l1)
    V_core = get_Vcore(l1,l2,h1,w1)
    return core_a * freq**core_x * B**core_y * core_LT * V_core

'''
Y1 = model1(X) # 1 : 0~50, 0 : 50~500
Y2 = model2(X) # 0~50 regression
return Y2*Y1 + (1-Y1)*999 # * : elemwise mult
'''


class MyProblem(Problem):
    def __init__(self,ML_model_num1,ML_model_num3,ML_model_num6,ML_model_num7):
        super().__init__(n_var=22,     #number of inputs
                         n_obj=2,     #number of outputs
                         n_constr=12,  #nubmer of constraints
                         # N1 N2 d1 d2 freq move_tx move_rx, offset_tx, offset_rx , per, space1, space2, space3, spae4, spae5, space6, l1, l2, h1, w1, I1, I2
                         xl=np.array([6, 8, 1.629, 1.629, 39.99e+3, 2.29,   2.29, -30, -30, 3499.9, 0, 5,  0, 5,  2.29, 2.29, 8,  10, 0,  20,  0.435, 0.316]), #input lower bounds
                         xu=np.array([16, 20, 1.631, 1.631, 40.01e+3, 5,  5, 30, 30,  3501.1,       7, 30, 7, 30, 2.39, 2.39, 20, 60, 80, 100, 0.437, 0.318]))   #input upper bounds
        self.ML_model_Lmt = ML_model_Lmt
        self.ML_model_Lmr = ML_model_Lmr
        self.ML_model_Llt = ML_model_Llt
        self.ML_model_Llr = ML_model_Llr
        self.ML_model_R1 = ML_model_R1
        self.ML_model_R2 = ML_model_R2
        self.ML_model_Lmtc = ML_model_Lmtc
        self.ML_model_Lmrc = ML_model_Lmrc
        self.ML_model_Lltc = ML_model_Lltc
        self.ML_model_Llrc = ML_model_Llrc
        self.ML_model_R1c = ML_model_R1c
        self.ML_model_R2c = ML_model_R2c


# ["N1","N2","d1","d2","freq","move_tx","move_rx","offset_tx","offset_rx","V1","per","space1","space2","space3","space4","l1", "l2", "h1", "w1", "I1", "I2"]
    def _evaluate(self, X, out, *args, **kwargs):


        N1 = X[:,0]
        N2 = X[:,1]

        freq = X[:,4]

        per = X[:,9] * 4 * math.pi * 1e-7

        # unit [m]
        d1 = X[:,2] * 1e-3
        d2 = X[:,3] * 1e-3

        move_tx = X[:,5] * 1e-3
        move_rx = X[:,6] * 1e-3

        offset_tx = X[:,7] * 1e-3
        offset_rx = X[:,8] * 1e-3
 
        space1 = X[:,10] * 1e-3
        space2 = X[:,11] * 1e-3 
        space3 = X[:,12] * 1e-3
        space4 = X[:,13] * 1e-3 
        space5 = X[:,14] * 1e-3
        space6 = X[:,15] * 1e-3 
        l1 = X[:,16] * 1e-3
        l2 = X[:,17] * 1e-3
        h1 = X[:,18] * 1e-3
        w1 = X[:,19] * 1e-3

        I1 = X[:,20]
        I2 = X[:,21]

        h11 = (N1/2)*move_tx + 2*d1
        h12 = (N2/2)*move_rx + 2*d2

        
        # Lmt
        f1 = self.ML_model_Lmt.predict( preprocess(X,new_feature_names=["N1s","offset","length1"]) )
        Lmt = f1 * 1e-3 # [unit : mH]
        f1 = f1/10

        # Lmr
        f2 = self.ML_model_Lmr.predict( preprocess(X,new_feature_names=["N2s","offset","length4"]) )
        Lmr = f2 * 1e-3 # [unit : mH]
        f2 = f2/10

        # total volume
        f3 = (h1 + 2*l1)*(4*l1+2*l2)*(w1+space4*2+space6*2+d2)   # Volume [Unit : m^3]

        # coreloss = core_a * freq^core_x * B^core_y * core_LT * V_core;
        V1 = 459.2
        mag_current = V1/(2*3.141592*freq)/Lmt
        Req = N1**2/Lmt
        flux = N1 * mag_current/Req
        B = flux / (2*w1*l1)
        V_core = ((h1 + 2*l1)*(4*l1+2*l2)*(w1) - 2*l2*h1*w1) # Core volue [unit : m^3]
        f4 = core_a * freq**core_x * B**core_y * core_LT * V_core # coreloss [unit : W]

        f5 = self.ML_model_R1.predict( preprocess(X,new_feature_names=["N1s","offset","length1"]) ) * I1**2 
        f6 = self.ML_model_R2.predict( preprocess(X,new_feature_names=["N2s","offset","length4"]) ) * I2**2

        f7 = f4+f5+f6

        f8 = self.ML_model_Llt.predict( preprocess(X,new_feature_names=["N1s","offset","length1","length4","window2","from_l2"]) )
        Llt = f8 * 1e-6

        #B = (1/N1)*(V1/2/3.141592/freq//Lmt)*(1/2/w1/l1) # (1/N1)*(V1/2/3.141592/freq//Lmt)*(1/2/w1/l1)

        consta = 10

        gLmt = (self.ML_model_Lmtc.predict(l2p(X)) - (1-1e-15)) * consta
        gLmr = (self.ML_model_Lmrc.predict(l2p(X)) - (1-1e-15)) * consta
        gLlt = (self.ML_model_Lltc.predict(l2p(X)) - (1-1e-15)) * consta
        gLlr = (self.ML_model_Llrc.predict(l2p(X)) - (1-1e-15)) * consta
        gR1 = (self.ML_model_R1c.predict(l2p(X)) - (1-1e-15)) * consta
        gR2 = (self.ML_model_R1c.predict(l2p(X)) - (1-1e-15)) * consta

        gN1 = -(N1/2%2 - 0.02)*(N1/2%2 - 0.98) * 0.1

        # Lmt constraint
        g1 = -(f1 - (5.0))
        # Lmr constraint
        g2 = -(Lmr/Lmt*1 - 1.7)*(Lmr/Lmt*1 - 2.1)/0.8
        g3 = -(N2/N1*1 - 1.32)*(N2/N1*1 - 1.42)/0.1

        gLlts = -(Llt * 0.9 - 40e-6)*(Llt * 1.1 - 40e-6) * 0.2/5e-6

        # length constraint
        #g3 = -(4*l1+2*l2 - (60e-3+2*space2))*(4*l1+2*l2 - 250*1e-3)

        const = 1

        # height constraint
        g4 = -(h1+2*l1 - ((N1-1)*move_tx+2*d1+10e-3))*(h1+2*l1 - 100e-3)/1e-3/50/const

        # width constraint
        #g5 = -(w1+space2+d2 - (50e-3+d2))*(w1+space2+d2 - (200*1e-3))

        # space constraint
        g6 = -(space1 - (d1/2+d2/2+0e-3))*(space1 - (d1/2+d2/2+5e-3))/1e-3/7/const
        g7 = -(space2 - (d1/2+d2/2+space1+space5+0.66e-3))*(space2 - (d2/2+space1+space5+20e-3))/1e-3/25/const
        g8 = -(space3 - (d1/2+d2/2+0e-3))*(space3 - (d1/2+d2/2+5e-3))/1e-3/7/const
        g9 = -(space4 - (d1/2+d2/2+space3+space6+0.66e-3))*(space4 - (d2/2+space3+space6+20e-3))/1e-3/25/const
        g10 = -(space5 - (d1+d2))*(space5 - (d1+d2+2e-3))/1e-3/5/10/const
        g11 = -(space6 - (d1+d2))*(space6 - (d1+d2+2e-3))/1e-3/5/10/const

        # offset constraint
        g12 = -(offset_tx - (-h1+h11)/2)*(offset_tx - (h1-h11)/2)/1e-3/60/const
        g13 = -(offset_rx - (-h1+h12)/2)*(offset_rx - (h1-h12)/2)/1e-3/60/const

        # core constraint
        g15 = -(l2 - (space2+space5+d2/2+3e-3))*(l2 - 60e-3)/1e-3/10/const
        g16 = -(h1 - h12)*(h1 - 80e-3)/1e-3/10/const


        #g5 = 1*X[:,7] + 1*X[:,3] + d2 - (160) # upper
        #g6 = 1*X[:,7] + 1*X[:,3] + d2 - (50 + d2) # lower0
        # l1 constraint
        #g7 = 1*X[:,4] - (50) # upper
        #g8 = 1*X[:,4] + (5) # lower
        # l2 constraint
        #g9 = 1*X[:,5] - (100) # upper
        #g10 = 1*X[:,5] + (1*X[:,3] + 20) # lower
        # h1 constraint
        #g11 = 1*X[:,6] - (200) # upper
        #g12 = 1*X[:,6] + (math.ceil((max(1*X[:,0],1*X[:,1])-1)*move_tx+2*d1)) # lower
        # w1 constraint
        #g13 = 1*X[:,7] - (200) # upper
        #g14 = 1*X[:,7] + (30) # lower
        # space1 constraint
        #g15 = 1*X[:,2] - (math.ceil(1*X[:,3]-d1-2)) # upper
        #g16 = 1*X[:,2] + (5) # lower
        # space2 constraint
        #g17 = 1*X[:,3] - (60) # upper
        #g18 = 1*X[:,3] + (20) # lower
        # N1 constraint
        #g19 = 1*X[:,1] // 1
        #g20 = 1*X[:,1] 


        #8.99995216e+00, 3.61173668e+00, 6.44412190e+01, 1.53430249e+01


        out["F"] = np.column_stack([f3, f7]) # "Minimize" values (volume, coreloss)
        out["G"] = np.column_stack([gN1,g4,g6,g7,g8,g9,g12,g13,g15,g16,f1,f2]) # > 0 Constraints
        out["G"] = - out["G"] # Actually < 0 
        out["H"] = np.column_stack([N1,N2,l1,l2,h1,w1,space1,space2,space3,space4,space5,space6,move_tx,move_rx,offset_tx,offset_rx])

problem = MyProblem(ML_model_Lmt, ML_model_Lmr, ML_model_R1, ML_model_R2)

In [7]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation

algorithm = NSGA2(
    pop_size=100,
    n_offsprings=20, # 10
    sampling=get_sampling("real_random"),
    crossover=get_crossover("real_sbx", prob=0.9, eta=15), #Simulated Binary Crossover
    mutation=get_mutation("real_pm", eta=40),
    eliminate_duplicates=True
)

In [8]:
from pymoo.factory import get_termination

termination = get_termination("n_gen", 1000)

In [9]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=False,
               verbose=True)
res

NameError: name 'csv' is not defined

In [ ]:
res

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(res.F[:,0]*1e+6, res.F[:,1])
plt.xlabel("volume [cm^3]")
plt.ylabel("loss [W]")
plt.grid(True)
#plt.ylim(0, 1000)

plt.rcParams["figure.figsize"] = (6,5)

In [ ]:
res.F[40]

In [ ]:
res.X[40]
Lmt[40]*1e+3
sol_copperloss_rx[40]+sol_copperloss_tx[40]+sol_coreloss[40]

In [ ]:
#plt.scatter(res.F[:,0]*1e+6, Lmr/Lmt)
plt.scatter((Lmr/Lmt), (N2/N1)**2)

Lmr

In [ ]:
# parameter calcaultion

sol_copperloss_tx = np.zeros(len(res.X))
sol_copperloss_rx = np.zeros(len(res.X))
sol_coreloss = np.zeros(len(res.X))

freq = np.zeros(len(res.X))
N1 = np.zeros(len(res.X))
N2 = np.zeros(len(res.X))
#Lmt = np.zeros(len(res.X))
#Lmr = np.zeros(len(res.X))
Llt = np.zeros(len(res.X))
Llr = np.zeros(len(res.X))
R1 = np.zeros(len(res.X))
R2 = np.zeros(len(res.X))
l1 = np.zeros(len(res.X))
l2 = np.zeros(len(res.X))
h1 = np.zeros(len(res.X))
w1 = np.zeros(len(res.X))

mag_current = np.zeros(len(res.X))
Req = np.zeros(len(res.X))
flux = np.zeros(len(res.X))
B = np.zeros(len(res.X))
V_core = np.zeros(len(res.X))

N1s = np.zeros(len(res.X))
N2s = np.zeros(len(res.X))
offset = np.zeros(len(res.X))
length1 = np.zeros(len(res.X))
length4 = np.zeros(len(res.X))
window2 = np.zeros(len(res.X))
from_l2 = np.zeros(len(res.X))


I1 = 4.36
I2 = 3.17

core_temp = 70

core_a = 0.6942
core_x = 1.4472
core_y = 2.4769
core_b = 4.7948
core_c = 0.0684
core_d = 4e-4
core_LT = core_b - core_c*core_temp + core_d*core_temp**2

Lmt = ML_model_Lmt.predict( preprocess(res.X,new_feature_names=["N1s","offset","length1"]) )
Lmr = ML_model_Lmr.predict( preprocess(res.X,new_feature_names=["N2s","offset","length4"]) )

for i in range(0,len(res.X)) :
    N1[i] = res.X[i][0]
    N2[i] = res.X[i][1]


'''
for i in range(0,len(res.X)) :

    param = [res.X[i]]
    param2 = pandas.DataFrame(param,columns=["N1","N2","d1","d2","freq","move_tx","move_rx","offset_tx","offset_rx","per","space1","space2","space3","space4","space5","space6","l1", "l2", "h1", "w1", "I1", "I2"])

    N1[i] = res.X[i][0]
    N2[i] = res.X[i][1]
    freq[i] = res.X[i][4]
    l1[i] = res.X[i][16]*1e-3
    l2[i] = res.X[i][17]*1e-3
    h1[i] = res.X[i][18]*1e-3
    w1[i] = res.X[i][19]*1e-3

    offset
    
    V1 = 459.2

    Lmt[i] = ML_model_Lmt.predict(param2)[0]*1e-3
    Lmr[i] = ML_model_Lmr.predict(param2)[0]*1e-3


    Llt[i] = ML_model_Llt.predict(param2)[0]*1e-6
    Llr[i] = ML_model_Llr.predict(param2)[0]*1e-6

    R1[i] = ML_model_R1.predict(param2)[0]
    R2[i] = ML_model_R2.predict(param2)[0]
    
    mag_current[i] = V1/(2*3.141592*freq[i])/Lmt[i]
    Req[i] = N1[i]**2/Lmt[i]
    flux[i] = N1[i] * mag_current[i]/Req[i]
    B[i] = flux[i] / (2*w1[i]*l1[i])
    V_core[i] = ((h1[i] + 2*l1[i])*(4*l1[i]+2*l2[i])*(w1[i]) - 2*l2[i]*h1[i]*w1[i]) # Core volue [unit : m^3]
    sol_coreloss[i] = core_a * freq[i]**core_x * B[i]**core_y * core_LT * V_core[i] # coreloss [unit : W]

    sol_copperloss_tx[[i]] = ML_model_R1.predict(param2)[0] * I1**2
    sol_copperloss_rx[[i]] = ML_model_R2.predict(param2)[0] * I2**2

'''

In [ ]:
# plot copper loss

plt.scatter(res.F[:,0]*1e+6, res.F[:,1])
plt.scatter(res.F[:,0]*1e+6, sol_copperloss_tx)
plt.scatter(res.F[:,0]*1e+6, sol_copperloss_rx)
plt.scatter(res.F[:,0]*1e+6, sol_copperloss_tx+sol_copperloss_rx)
plt.scatter(res.F[:,0]*1e+6, sol_coreloss)
plt.xlabel("volume [cm^3]")
plt.ylabel("loss [W]")
plt.grid(True)

plt.legend(["total loss (copper + core)","primary side copper loss","secondary side copper loss","total copper loss","core loss"])

plt.rcParams["figure.figsize"] = (6,5)

In [ ]:
data_Lmt = pandas.DataFrame(Lmt*1e+3,columns=["Lmt"])
data_F0 = pandas.DataFrame(res.F[:,0],columns=["total_volume"])
data_F1 = pandas.DataFrame(res.F[:,1],columns=["total_loss"])

data_Lmt_copper = pandas.concat([data_Lmt,data_F0,data_F1],axis=1)
data_Lmt_copper = data_Lmt_copper[data_Lmt_copper['total_loss']<5000].reset_index(drop=True)

#plt.scatter(data_Lmt_copper['total_volume']*1e+6, data_Lmt_copper['total_loss'])
plt.scatter(data_Lmt_copper['total_loss'], data_Lmt_copper['Lmt'])

plt.xlabel("total loss [W]")
plt.ylabel("inductance [mH]")
plt.grid(True)

plt.rcParams["figure.figsize"] = (6,5)

In [ ]:
data_Llt = pandas.DataFrame(Llt*1e+6,columns=["Llt"])
data_F0 = pandas.DataFrame(res.F[:,0],columns=["total_volume"])
data_F1 = pandas.DataFrame(res.F[:,1],columns=["total_loss"])

data_Llt_copper = pandas.concat([data_Llt,data_F0,data_F1],axis=1)
data_Llt_copper = data_Llt_copper[data_Llt_copper['total_loss']<5000].reset_index(drop=True)

#plt.scatter(data_Lmt_copper['total_volume']*1e+6, data_Lmt_copper['total_loss'])
plt.scatter(data_Llt_copper['total_loss'], data_Llt_copper['Llt'])

plt.xlabel("total loss [W]")
plt.ylabel("leakage inductance [uH]")
plt.grid(True)

plt.rcParams["figure.figsize"] = (6,5)

In [ ]:
Llt*1e+6

In [ ]:
#data Save

data_X = pandas.DataFrame(res.X,columns=["N1","N2","d1","d2","freq","move_tx","move_rx","offset_tx","offset_rx","per","space1","space2","space3","space4","space5","space6","l1", "l2", "h1", "w1", "I1", "I2"])
data_F0 = pandas.DataFrame(res.F[:,0]*1e+6,columns=["volume"])
data_F1 = pandas.DataFrame(res.F[:,1],columns=["total_loss"])
data_Lmt = pandas.DataFrame(Lmt*1e+3,columns=["Lmt"])
data_Lmr = pandas.DataFrame(Lmr*1e+3,columns=["Lmr"])
data_Llt = pandas.DataFrame(Llt*1e+6,columns=["Llt"])
data_Llr = pandas.DataFrame(Llr*1e+6,columns=["Llr"])
data_R1 = pandas.DataFrame(R1,columns=["R1"])
data_R2 = pandas.DataFrame(R2,columns=["R2"])
data_copperloss_tx = pandas.DataFrame(sol_copperloss_tx,columns=["copperloss_tx"])
data_copperloss_rx = pandas.DataFrame(sol_copperloss_rx,columns=["copperloss_rx"])
data_copperloss = pandas.DataFrame(sol_copperloss_tx+sol_copperloss_rx,columns=["copperloss"])
data_coreloss = pandas.DataFrame(sol_coreloss,columns=["coreloss"])
savedata = pandas.concat([data_X,data_F0,data_F1,data_Lmt,data_Lmr,data_Llt,data_Llr,data_R1,data_R2,data_copperloss_tx,data_copperloss_rx,data_copperloss,data_coreloss],axis=1)

savedata

#savedata.to_csv("./NSGA2_data_2021_08_12_v2_N1000.csv")

In [ ]:
res.X[50]

In [ ]:
res.X[15]

In [ ]:
idx = 5
rtrt = res.F[idx,0]
vvvv = res.F[idx,1]
inin = res.X[idx]
ouou = -res.G[idx]
print(f'volume = {rtrt*1e-3} \ncoreloss={vvvv*1e+9} \nInputs={inin}\n[n1, n2, space1, space2, l1, l2, h1, w1] \nOutputs={ouou}')